## Project: Predicting Boston Housing Prices

## Getting Started
In this project, the performance and predictive power of a model that has been trained and tested on data collected from homes in suburbs of Boston, Massachusetts are evaluated. A model trained on this data that is seen as a *good fit* could then be used to make certain predictions about a home — in particular, its monetary value. This model would prove to be invaluable for someone like a real estate agent who could make use of such information on a daily basis.

The dataset for this project originates from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Housing). The Boston housing data was collected in 1978 and each of the 506 entries represent aggregated data about 14 features for homes from various suburbs in Boston, Massachusetts. For the purposes of this project, the following preprocessing steps have been made to the dataset:
- 16 data points have an `'MEDV'` value of 50.0. These data points likely contain **missing or censored values** and have been removed.
- 1 data point has an `'RM'` value of 8.78. This data point can be considered an **outlier** and has been removed.
- The features `'RM'`, `'LSTAT'`, `'PTRATIO'`, and `'MEDV'` are essential. The remaining **non-relevant features** have been excluded.
- The feature `'MEDV'` has been **multiplicatively scaled** to account for 35 years of market inflation.

In [10]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit

# Import supplementary visualizations code visuals.py
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

# Load the Boston housing dataset
data = pd.read_csv('./housing.csv')
prices = data['MEDV']
features = data.drop('MEDV', axis = 1)

print (f"Boston housing dataset has {data.shape[0]} data points with {data.shape[1]} variables each.")#.format(*data.shape)

Boston housing dataset has 489 data points with 4 variables each.


## Data Exploration
A cursory investigation about the Boston housing data is made.

Since the main goal of this project is to construct a working model which has the capability of predicting the value of houses, the dataset needs to be separated into **features** and the **target variable**. The **features**, `'RM'`, `'LSTAT'`, and `'PTRATIO'`, give quantitative information about each data point. The **target variable**, `'MEDV'`, will be the variable to be predicted. These are stored in `features` and `prices`, respectively.

### Descriptive Statistics

In [20]:
# Minimum price of the data
minimum_price = np.min(prices)

# Maximum price of the data
maximum_price = np.max(prices)

# Mean price of the data
mean_price = np.mean(prices)

# Median price of the data
median_price = np.median(prices)

# Standard deviation of prices of the data
std_price = np.std(prices)

# Print the calculated statistics
print ("Statistics for Boston housing dataset:\n")
print (f"Minimum price: ${minimum_price:,.{9}}") 
print (f"Maximum price: ${maximum_price:,.{9}}") 
print (f"Mean price: ${mean_price:,.{9}}") 
print (f"Median price ${median_price:,.{9}}") 
print (f"Standard deviation of prices: ${std_price:,.{9}}") 

Statistics for Boston housing dataset:

Minimum price: $105,000.0
Maximum price: $1,024,800.0
Mean price: $454,342.945
Median price $438,900.0
Standard deviation of prices: $165,171.132


### Feature Observation
Three features from the Boston housing dataset are used: `'RM'`, `'LSTAT'`, and `'PTRATIO'`. For each data point (neighborhood):
- `'RM'` is the average number of rooms among homes in the neighborhood.
- `'LSTAT'` is the percentage of homeowners in the neighborhood considered "lower class" (working poor).
- `'PTRATIO'` is the ratio of students to teachers in primary and secondary schools in the neighborhood.

RM: I would expect an increase in RM to lead to an increase in MEDV. This is based on my intuition that a 5-bedroom house would cost more than a 4-bedroom house.  
LSTAT: An increase in LSTAT would probably lead to a decrease in MEDV. This can be related to the assumption that the rise in income of homeowners would drive the prices up.  
PTRATIO: A higher PTRATIO would lead to a decrease in MEDV. For example, a school with 1 teacher for every 10 students, might perform better than a school with 1 teacher for every 15 students, since the teachers would have more bandwidth for each student.  

## Developing a Model

### Performance Metric
It is difficult to measure the quality of a given model without quantifying its performance over training and testing. This is typically done using some type of performance metric, whether it is through calculating some type of error, the goodness of fit, or some other useful measurement. Here, [*coefficient of determination*](http://stattrek.com/statistics/dictionary.aspx?definition=coefficient_of_determination), R<sup>2</sup> is used to quantify the model's performance. The coefficient of determination for a model describes how "good" that model is at making predictions. 

The values for R<sup>2</sup> range from 0 to 1, which captures the percentage of squared correlation between the predicted and actual values of the **target variable**. A model with an R<sup>2</sup> of 0 is no better than a model that always predicts the *mean* of the target variable, whereas a model with an R<sup>2</sup> of 1 perfectly predicts the target variable. Any value between 0 and 1 indicates what percentage of the target variable, using this model, can be explained by the **features**. _A model can be given a negative R<sup>2</sup> as well, which indicates that the model is **arbitrarily worse** than one that always predicts the mean of the target variable._

In [21]:
# Import 'r2_score'
from sklearn.metrics import r2_score

def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # Calculate the performance score between 'y_true' and 'y_predict'
    score = r2_score(y_true, y_predict)
    
    # Return the score
    return score

### Shuffle and Split Data
The Boston housing dataset is split into training and testing subsets. Typically, the data is also shuffled into a random order when creating the training and testing subsets to remove any bias in the ordering of the dataset.

In [23]:
# Import 'train_test_split'
from sklearn.model_selection import train_test_split

# TODO: Shuffle and split the data into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(features, prices, test_size=0.2, random_state=42)

# Success
print ("Training and testing split was successful.")

Training and testing split was successful.


### Training and Testing

Splitting data into training and testing sets allows us to first train the model on a subset of the data (training set) and build a generalized model. Then we can use this model on another subset of the data (testing set) and since the testing data is labelled (in our case), then we can compare how good our predictions are with the true target value.  
Underfitting happens when the model is too simple and cannot generalize for new data. Overfitting happens when the model learns the data too much (memorizes it) and will perform great on the training set, but will not be able to generalize for new data.  
When we split data into training and testing, ensuring that the split is random helps in not predicting targets just because of the order of data. This would minimize underfitting (maybe the model is too simple by just looking at the order of the data) and overfitting.

### Fitting a Model
A model is trained using the **decision tree algorithm**. To ensure that you are producing an optimized model, grid search technique to optimize the `'max_depth'` parameter for the decision tree. The `'max_depth'` parameter can be thought of as how many questions the decision tree algorithm is allowed to ask about the data before making a prediction. Decision trees are part of a class of algorithms called *supervised learning algorithms*.

In addition, `ShuffleSplit()` is used for an alternative form of cross-validation. The `ShuffleSplit()` implementation below will create 10 (`'n_splits'`) shuffled sets, and for each shuffle, 20% (`'test_size'`) of the data will be used as the *validation set*. 

In [24]:
# Import 'make_scorer', 'DecisionTreeRegressor', and 'GridSearchCV'
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV


def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # sklearn version 0.18: ShuffleSplit(n_splits=10, test_size=0.1, train_size=None, random_state=None)
    # sklearn versiin 0.17: ShuffleSplit(n, n_iter=10, test_size=0.1, train_size=None, random_state=None)
    #cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)
    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.20, random_state = 0)

    # Create a decision tree regressor object
    regressor = DecisionTreeRegressor(random_state=42)

    # Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'max_depth': range(1,11)}

    # Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(performance_metric)

    # Create the grid search cv object --> GridSearchCV()
    grid = GridSearchCV(regressor, params, scoring=scoring_fnc, cv=cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

### Making Predictions
Once a model has been trained on a given set of data, it can now be used to make predictions on new sets of input data. In the case of a *decision tree regressor*, the model has learned *what the best questions to ask about the input data are*, and can respond with a prediction for the **target variable**. These predictions can be used to gain information about data where the value of the target variable is unknown — such as data the model was not trained on.

### Optimal Model

In [25]:
# Fit the training data to the model using grid search
reg = fit_model(X_train, y_train)

# Produce the value for 'max_depth'
print (f"Parameter 'max_depth' is {reg.get_params()['max_depth']} for the optimal model.")

Parameter 'max_depth' is 4 for the optimal model.


### Predicting Selling Prices
For sample test data below, prediction is made to price the homes.

| Feature | Client 1 | Client 2 | Client 3 |
| :---: | :---: | :---: | :---: |
| Total number of rooms in home | 5 rooms | 4 rooms | 8 rooms |
| Neighborhood poverty level (as %) | 17% | 32% | 3% |
| Student-teacher ratio of nearby schools | 15-to-1 | 22-to-1 | 12-to-1 |

Of the three clients, client 3 has has the biggest house, in the best public school neighborhood with the lowest poverty level; while client 2 has the smallest house, in a neighborhood with a relatively high poverty rate and not the best public schools.

In [30]:
# Produce a matrix for client data
client_data = [[5, 17, 15], # Client 1
               [4, 32, 22], # Client 2
               [8, 3, 12]]  # Client 3

# Show predictions
for i, price in enumerate(reg.predict(client_data)):
    print (f"Predicted selling price for Client {i+1}'s home: ${price:,.{8}}")

Predicted selling price for Client 1's home: $403,025.0
Predicted selling price for Client 2's home: $237,478.72
Predicted selling price for Client 3's home: $931,636.36


These predictions are in line with intuitions.  
- Client 2 has a low sale price, closer to the minimum price of 105,000.00 USD. This relates to the small house size, high poverty rate, and bad schools.  
- Client 3 has a high sale price, closer to the maximum price of 1,024,800.00 USD. This relates to the big house size, very low poverty rate and good schools.  
- Client 1 is at the middle of the sale price range (mean and median sale prices are in the 400k USD range).

### Sensitivity
An optimal model is not necessarily a robust model. Sometimes, a model is either too complex or too simple to sufficiently generalize to new data. Sometimes, a model could use a learning algorithm that is not appropriate for the structure of the data given. Other times, the data itself could be too noisy or contain too few samples to allow a model to adequately capture the target variable — i.e., the model is underfitted. 

**The code below runs the `fit_model` function ten times with different training and testing sets to see how the prediction for a specific client changes with respect to the data it's trained on.**

In [35]:
vs.PredictTrials(features, prices, fit_model, client_data)

Trial 1: $3.9e+05
Trial 2: $4.2e+05
Trial 3: $4.2e+05
Trial 4: $4.2e+05
Trial 5: $4.2e+05
Trial 6: $4.1e+05
Trial 7: $4e+05
Trial 8: $4.1e+05
Trial 9: $3.5e+05
Trial 10: $4.1e+05

Range in prices: $6.9e+04


### Applicability

I would not advise using the contructed model in a real-world setting. Here are my reasons:  

- Since the original data is old (1978) and was adjusted for inflation, it might not reflect prices accurately. Inflation is important, but there are other factors too such as how the area has changed, if it is no longer considered a residential area, or if it has evolved into an industrial or commercial area.  
- There are only 3 features used to describe a home, which is certainly insufficient. Number of rooms is not indicative of the actual size of the house. There are several other features that would be necessary like lot area, year the house was built, appliances, number of storeys, remodelling (if house is old), presence of termites and other pests, and so on.  
- As we can see from the Sensitivity of the model, there is a \$69000 range in the predictions. This is a significant variance. So I would not consider the model robust enough.  
- Data from an urban city would not apply to a rural city, where the prices might be lower.  
- Even though neighborhood quality would affect the price of an individual home, sometimes it might be unfair to give it too much importance if the home fairs very well in other factors.